In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[9],7
commitnumber,1013f91,5014a19
conc,-,1.8e-06
dv,-,0.001
klin,-,2e-21
molecule,{'ch4': 1.8e-06},ch4
ng_adju,-,"[0, 0]"
ng_refs,-,"[2, 3]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.048886                    1 -0.008027
0.000750        2 -0.044636                    2 -0.005460
0.001052        3 -0.035353                    3  0.000602
0.001476        4 -0.025072                    4  0.007149
0.002070        5 -0.013849                    5  0.014386
0.002904        6 -0.001629                    6  0.022324
0.004074        7  0.011642                    7  0.030982
0.005714        8  0.026034                    8  0.040512
0.008015        9  0.041584                    9  0.050919
0.011243       10  0.058361                   10  0.062273
0.015771       11  0.076396                   11  0.074645
0.022122       12  0.095713                   12  0.088102
0.031031       13  0.116256                   13  0.102653
0.043528       14  0.135205                   14  0.116484
0.061057       15  0.149673                   15  0.127794
0.085645       16  0.161735                   16  0.138007
0.120136       17  0.173516                   17  0.148585
0.168516       18  0.185014                   18  0.159567
0.236378       19  0.196248                   19  0.170887
0.331549       20  0.207499                   20  0.182633
0.465100       21  0.217729                   21  0.193403
0.652400       22  0.198993                   22  0.183011
0.915100       23  0.146774                   23  0.146865
1.283650       24  0.096346                   24  0.108372
1.800600       25  0.059257                   25  0.075748
2.525700       26  0.033524                   26  0.048666
3.542800       27  0.016602                   27  0.027029
4.969550       28  0.008060                   28  0.012709
6.970850       29  0.005762                   29  0.005918
9.778100       30  0.005070                   30  0.003142
13.715850      31  0.004063                   31  0.001971
19.239350      32  0.002658                   32  0.001440
26.987250      33  0.002382                   33  0.002267
37.855300      34  0.003332                   34  0.003987
53.100050      35  0.003867                   35  0.004986
73.887500      36  0.003960                   36  0.005338
97.662500      37  0.003649                   37  0.005125
121.437500     38  0.002439                   38  0.003915
145.212500     39  0.000708                   39  0.002016
168.987500     40 -0.000969                   40 -0.000034
192.762500     41 -0.002721                   41 -0.002303
216.537500     42 -0.004641                   42 -0.004860
240.312500     43 -0.006852                   43 -0.007743
264.087500     44 -0.009578                   44 -0.011054
287.862500     45 -0.012887                   45 -0.014589
311.637500     46 -0.014531                   46 -0.016330
335.412500     47 -0.013063                   47 -0.015158
359.187500     48 -0.010524                   48 -0.012716
382.962500     49 -0.008206                   49 -0.010244
406.737500     50 -0.005959                   50 -0.007779
430.512500     51 -0.003671                   51 -0.005281
454.287500     52 -0.001290                   52 -0.002748
478.062500     53  0.001207                   53 -0.000087
501.837500     54  0.003822                   54  0.002631
525.612500     55  0.006543                   55  0.005432
549.387500     56  0.009336                   56  0.008285
573.162500     57  0.012153                   57  0.011145
596.937500     58  0.014934                   58  0.013953
620.712500     59  0.017616                   59  0.016617
644.487500     60  0.020150                   60  0.019201
668.262500     61  0.022510                   61  0.021590
692.037500     62  0.024702                   62  0.023825
715.812500     63  0.026765                   63  0.025952
739.587500     64  0.028762                   64  0.028036
763.362500     65  0.030772                   65  0.030141
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -8.185270  0.000000 -8.185270 -8.178177  9.400515e-08   
0.000624    2     -8.185276  0.000003 -8.185273 -8.178180  1.946683e-06   
0.000876    3     -8.185279  0.000004 -8.185275 -8.178181  2.738895e-06   
0.001229    4     -8.185282  0.000006 -8.185276 -8.178182  3.978909e-06   
0.001723    5     -8.185286  0.000009 -8.185278 -8.178183  5.909889e-06   
0.002417    6     -8.185292  0.000013 -8.185279 -8.178185  8.915985e-06   
0.003391    7     -8.185299  0.000020 -8.185279 -8.178187  1.359191e-05   
0.004757    8     -8.185308  0.000031 -8.185277 -8.178190  2.085360e-05   
0.006672    9     -8.185319  0.000048 -8.185271 -8.178192  3.212341e-05   
0.009359    10    -8.185332  0.000074 -8.185258 -8.178193  4.959221e-05   
0.013128    11    -8.185346  0.000115 -8.185232 -8.178192  7.663514e-05   
0.018415    12    -8.185361  0.000176 -8.185184 -8.178187  1.184437e-04   
0.025830    13    -8.185371  0.000271 -8.185100 -8.178174  1.829854e-04   
0.036232    14    -8.185373  0.000416 -8.184957 -8.178147  2.824241e-04   
0.050823    15    -8.185358  0.000635 -8.184723 -8.178097  4.336837e-04   
0.071291    16    -8.185316  0.000955 -8.184360 -8.178013  6.591312e-04   
0.100000    17    -8.185230  0.001420 -8.183810 -8.177876  9.917601e-04   
0.140271    18    -8.185072  0.002090 -8.182982 -8.177657  1.481470e-03   
0.196760    19    -8.184791  0.003046 -8.181744 -8.177308  2.200642e-03   
0.275997    20    -8.184304  0.004402 -8.179902 -8.176756  3.252973e-03   
0.387100    21    -8.183473  0.006301 -8.177171 -8.175885  4.785553e-03   
0.543100    22    -8.182064  0.008916 -8.173148 -8.174520  6.994458e-03   
0.761700    23    -8.180073  0.012078 -8.167995 -8.172657  9.871721e-03   
1.068500    24    -8.177860  0.015199 -8.162661 -8.170558  1.311069e-02   
1.498800    25    -8.175639  0.017889 -8.157750 -8.168383  1.645959e-02   
2.102400    26    -8.173483  0.019970 -8.153513 -8.166205  1.969803e-02   
2.949000    27    -8.171500  0.021349 -8.150151 -8.164139  2.251342e-02   
4.136600    28    -8.169886  0.022070 -8.147816 -8.162370  2.454732e-02   
5.802500    29    -8.168746  0.022521 -8.146225 -8.160994  2.567995e-02   
8.139200    30    -8.167970  0.023340 -8.144630 -8.159921  2.624524e-02   
11.417000   31    -8.167599  0.024937 -8.142662 -8.159212  2.675628e-02   
16.014700   32    -8.168038  0.027590 -8.140449 -8.159305  2.792220e-02   
22.464000   33    -8.170210  0.031792 -8.138418 -8.161095  3.081258e-02   
31.510500   34    -8.174867  0.039002 -8.135865 -8.165285  3.743297e-02   
44.200100   35    -8.182267  0.051410 -8.130857 -8.172138  5.027997e-02   
62.000000   36    -8.193547  0.070844 -8.122704 -8.182821  7.147698e-02   
85.775000   37    -8.209869  0.098317 -8.111551 -8.198617  1.023090e-01   
109.550000  38    -8.227569  0.126294 -8.101274 -8.216140  1.342654e-01   
133.325000  39    -8.247599  0.153194 -8.094404 -8.236472  1.656251e-01   
157.100000  40    -8.271002  0.178591 -8.092411 -8.260717  1.955489e-01   
180.875000  41    -8.298090  0.202949 -8.095140 -8.289291  2.240272e-01   
204.650000  42    -8.329257  0.226454 -8.102804 -8.322734  2.509841e-01   
228.425000  43    -8.365085  0.249211 -8.115874 -8.361816  2.763764e-01   
252.200000  44    -8.406472  0.271301 -8.135172 -8.407464  3.002143e-01   
275.975000  45    -8.454950  0.292803 -8.162147 -8.460969  3.225855e-01   
299.750000  46    -8.512747  0.314307 -8.198440 -8.523520  3.440466e-01   
323.525000  47    -8.578897  0.339532 -8.239365 -8.593266  3.677957e-01   
347.300000  48    -8.649009  0.372855 -8.276154 -8.666182  3.980162e-01   
371.075000  49    -8.720447  0.414654 -8.305793 -8.739920  4.359401e-01   
394.850000  50    -8.792516  0.463614 -8.328902 -8.813773  4.809408e-01   
418.625000  51    -8.864684  0.519001 -8.345684 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')